In [ ]:
def gen_barcode_mrid(patternLengths, c2c_dists, barcode_length=4500, barcode_start=1000):
    
    barcode = np.ones((1000,barcode_length))
    l=barcode_start
    ticks=[l]
        
    for i, bar in enumerate(patternLengths):
        if l==barcode_start:
            barInt=np.round(bar).astype(int)
            barcode[:, l:l+barInt]=0
            ticks.append(l+barInt/2)
            ticks.append(l+barInt)
            l=l+np.round(bar/2).astype(int)+np.round(c2c_dists[i]).astype(int)
        else:
            barInt=np.round(bar/2).astype(int)
            barcode[:, l-barInt:l+barInt]=0
            ticks.append(int(l-barInt))
            ticks.append(l)
            ticks.append(l+barInt)
            if i<len(c2c_dists):
                l=l+np.round(c2c_dists[i]).astype(int)
                
    ticks=np.array(ticks)
    tickLabels=(ticks-barcode_start).astype(int)
    
    return barcode, ticks, tickLabels

def gen_barcode_wafer(patternPoints, barcode_start=1000, barcode_length=4500):
    num_patterns=int(len(patternPoints)/3)
    barcode = np.ones((1000,barcode_length))
    l=barcode_start

    for pattern in range(num_patterns):
        p1,p2,p3=patternPoints[3*pattern:3*(pattern+1)]
        barcode[:, int(l+p1):int(l+p3+1)]=0

    return barcode

def equalize_barcodes(barcode1, barcode2):
    if np.shape(barcode1)[1]>np.shape(barcode2)[1]:
        diff=np.shape(barcode1)[1]-np.shape(barcode2)[1]
        padd=np.ones((np.shape(barcode2)[0], diff))
        barcode2=np.concatenate((padd, barcode2), axis=1)
    elif np.shape(barcode1)[1]<np.shape(barcode2)[1]:
        diff=np.shape(barcode2)[1]-np.shape(barcode1)[1]
        padd=np.ones((np.shape(barcode1)[0], diff))
        barcode1=np.concatenate((padd, barcode1), axis=1)
    
    return barcode1, barcode2
def barcode_cc(barcode_fix, barcode_mvng):
#     barcode_fix, barcode_mvng = equalize_barcodes(barcode_fix, barcode_mvng)
    stride=1
    cc=[]
    barcode_fix=1-barcode_fix
    barcode_fix=barcode_fix[0,:]
#     print("zero padding: "+str(np.shape(barcode_mvng)[-1]))
    barcode_fix=np.append(np.zeros((np.shape(barcode_mvng)[-1])), barcode_fix)
    barcode_fix=np.append(barcode_fix, np.zeros((np.shape(barcode_mvng)[-1])))
    
    barcode_mvng=1-barcode_mvng
    barcode_mvng=barcode_mvng[0,:]
    for i in range(len(barcode_fix)-len(barcode_mvng)):
        corr=np.multiply(barcode_fix[stride*i:stride*i+len(barcode_mvng)], barcode_mvng)
        cc.append(np.sum(corr))    
    
    return cc